# Fashion Concierge – Interactive Demo Notebook

This notebook is an **interactive front end** for the Fashion Concierge project built with the
**Google Agent Development Kit (ADK)** and **Gemini**.

It is designed to work for **any user** who has access to the repository, whether they run it:

- in **GitHub Codespaces**, or  
- in a local Python environment (VS Code, Jupyter Lab, etc.).

The notebook demonstrates three core capabilities:

1. Wiring the notebook to the **Fashion Concierge backend** (the `FashionConciergeApp` class in `adk_app/app.py`).  
2. Running **one end-to-end outfit suggestion** through the agent pipeline.  
3. Showing a **simple session and memory example** where the agent remembers user preferences.


## 0. How to use this notebook

### 0.1 Open the notebook in GitHub Codespaces

1. Navigate to the Fashion Concierge repository on GitHub.  
2. Click **Code → Open with Codespaces → New codespace**.  
3. When the Codespace finishes starting, open this notebook file (for example `notebooks/fashion_concierge_demo.ipynb`) in the editor.  
4. Select the default Python kernel for the Codespace.

> You can also run this notebook locally if you prefer. The steps are the same after cloning the repo.

### 0.2 Install dependencies

Inside the Codespace (or your local terminal), run once:

```bash
pip install -e .
```

### 0.3 Configure credentials

Make sure the environment has credentials for:

- Google Gemini / Vertex AI (for the LLM calls).  
- Any external tools you actually use (calendar, weather, etc.), if those are enabled.

In many setups this is handled via environment variables (`GOOGLE_API_KEY`, `GOOGLE_CLOUD_PROJECT`, etc.).
The next section will show where these are read.


## 1. Environment configuration

In [ ]:
"""Environment and path setup.

This cell:

1. Locates the repository root (so imports work whether the notebook lives in
   the root or in a subfolder like `notebooks/`).
2. Adds the repo root to `sys.path`.
3. Sets **placeholder** environment variables for Google Cloud / Gemini so that
   the rest of the code can read them. In Codespaces or local dev you should
   either:
   - export real values before starting Jupyter, or
   - edit the placeholders below.
"""

import os
import sys
from pathlib import Path

# --- Locate the project root --------------------------------------------------
# We look upwards from the notebook directory until we find the marker folder
# `adk_app` (which exists in this repo) or a `.git` directory.
current = Path().resolve()
project_root = None

for parent in [current] + list(current.parents):
    if (parent / "adk_app").exists() or (parent / ".git").exists():
        project_root = parent
        break

if project_root is None:
    raise RuntimeError(
        "Could not locate the project root. Make sure you are running this "
        "notebook inside the Fashion Concierge repository."
    )

print("Detected project root:", project_root)

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Sanity check
if not (project_root / "adk_app").exists():
    raise RuntimeError(
        "The 'adk_app' package was not found at the detected project root. "
        "Please confirm the repo layout."
    )
else:
    print("Found 'adk_app' package. Imports should work.")

# --- Environment variables for Google / tools ---------------------------------
# These are **placeholders**. Replace with real values or rely on values
# already exported in the environment. Using `setdefault` means they will
# not override anything you configured outside the notebook.
os.environ.setdefault("GOOGLE_CLOUD_PROJECT", "GOOGLE_CLOUD_PROJECT")
os.environ.setdefault("GOOGLE_CLOUD_LOCATION", "GOOGLE_CLOUD_LOCATION")  # e.g. "europe-west4"
os.environ.setdefault("GOOGLE_API_KEY", "GOOGLE_API_KEY")

print("GOOGLE_CLOUD_PROJECT:", os.environ.get("GOOGLE_CLOUD_PROJECT"))
print("GOOGLE_CLOUD_LOCATION:", os.environ.get("GOOGLE_CLOUD_LOCATION"))

Detected project root: /workspaces/FashionConcierge
Found 'adk_app' package. Imports should work.
GOOGLE_CLOUD_PROJECT: fashion-concierge-0
GOOGLE_CLOUD_LOCATION: europe-west4


## 2. Load the Fashion Concierge backend

The Fashion Concierge backend is wrapped in a convenience class `FashionConciergeApp`
defined in `adk_app/app.py`.

This class is responsible for:

- Constructing the ADK `App` object.  
- Registering all agents (calendar, weather, wardrobe, stylist, critic, etc.).  
- Exposing high-level methods for:
  - creating sessions  
  - orchestrating outfit planning  
  - running conversations

We import and instantiate it here.

> If your repository uses a different entry point (for example a `create_app()`
> function), you only need to adapt the import below.


In [2]:
from adk_app.app import FashionConciergeApp  # adapt if your module name differs

fashion_app = FashionConciergeApp()

print("FashionConciergeApp instance:", fashion_app)
print("Public attributes:", [a for a in dir(fashion_app) if not a.startswith("_")])

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FashionConciergeApp instance: <adk_app.app.FashionConciergeApp object at 0x73f7b84b9fa0>
Public attributes: ['adk_app', 'calendar_agent', 'calendar_provider', 'config', 'ingestion_tool_defs', 'memory_service', 'memory_tool_defs', 'orchestrator', 'outfit_stylist', 'quality_critic', 'send_test_message', 'session_manager', 'session_store', 'session_tool_defs', 'start_session', 'wardrobe_ingestion', 'wardrobe_query', 'wardrobe_store', 'wardrobe_tool_defs', 'wardrobe_tools', 'weather_agent', 'weather_provider']


### 2.1 Helper functions for sessions and outfit orchestration

The HTTP API layer (in `server/api.py`) usually calls methods on `FashionConciergeApp`
to:

- create a new session, and  
- orchestrate an outfit recommendation.

To make the notebook easier to read, we mirror that behaviour with two helper
functions:

- `create_demo_session(...)`  
- `orchestrate_outfit(...)`  

If the method names in your implementation differ, you can adjust **only this
section** and the rest of the notebook will still work.


In [16]:
from typing import Optional, Dict, Any

def create_demo_session(
    metadata: Optional[Dict[str, Any]] = None,
    user_id: str = "notebook-demo-user",
):
    """Create a new session through `FashionConciergeApp`.

    Uses `start_session(user_id=..., metadata=...)`.

    In this implementation `start_session` returns the session id directly
    (as a string), so we treat that as both the "session object" and the id.
    """
    if not hasattr(fashion_app, "start_session"):
        raise AttributeError(
            "FashionConciergeApp has no 'start_session' method. "
            "Open adk_app/app.py and either expose one or adapt this helper."
        )

    session = fashion_app.start_session(
        user_id=user_id,
        metadata=metadata or {"source": "notebook-demo"},
    )

    # In this app, start_session returns the session id directly
    session_id = str(session)

    print("Created session id:", session_id)
    return session, session_id


def send_demo_message(
    session_id: str,  # kept for symmetry, but not used by this helper
    user_query: str,
) -> Any:
    """Call the FashionConciergeApp's test message entry point.

    This is a simple end to end smoke test that routes to the orchestrator.
    It is useful for checking wiring and sessions.

    Note: the current implementation of `send_test_message` on FashionConciergeApp
    does not accept a session_id parameter, so we ignore `session_id` here.
    """
    if not hasattr(fashion_app, "send_test_message"):
        raise AttributeError(
            "FashionConciergeApp has no 'send_test_message' method. "
            "Open adk_app/app.py and either expose one or adapt this helper."
        )

    # Only pass the text, as send_test_message(user_query) is the actual signature
    return fashion_app.send_test_message(user_query)


def orchestrate_outfit(
    session_id: str,
    user_query: str,
    location: str = "Amsterdam, NL",
    date_iso: Optional[str] = None,
    mood: Optional[str] = None,
) -> Any:
    """Temporary high-level 'plan outfit' call.

    At the moment `FashionConciergeApp` does not expose a dedicated
    `orchestrate_outfit` or `plan_outfit` method. Until that exists, we
    reuse `send_test_message` and enrich the user query with lightweight
    context hints (location, date, mood).

    Once the backend exposes a proper orchestration method, replace the body
    of this function with a direct call to that method.
    """
    if not hasattr(fashion_app, "send_test_message"):
        raise AttributeError(
            "FashionConciergeApp has no 'send_test_message' method. "
            "Open adk_app/app.py and either expose one or adapt this helper."
        )

    # Compose a richer prompt with context hints. The orchestrator stub can
    # then use this text if needed.
    context_hint_parts = []
    if location:
        context_hint_parts.append(f"location={location}")
    if date_iso:
        context_hint_parts.append(f"date={date_iso}")
    if mood:
        context_hint_parts.append(f"mood={mood}")

    context_hint = ", ".join(context_hint_parts) if context_hint_parts else "none"

    full_prompt = (
        f"{user_query}\n\n"
        f"[Context hint from notebook: {context_hint}]"
    )

    # Note: send_test_message currently does not take a session id parameter.
    # We ignore `session_id` here until the backend provides a session-aware
    # orchestrator entry point.
    return fashion_app.send_test_message(full_prompt)

## 2.B Wardrobe ingestion and inspection

This section shows how to add items to the wardrobe store and inspect what is
available for a given user. It supports ingestion from product links and from
tabular data in a pandas DataFrame.


In [ ]:
import pandas as pd

user_id = "notebook-demo-user"  # keep consistent with other notebook sections

# Replace these placeholders with real retailer product URLs before running.
product_urls = [
    "https://www.example.com/product-1",
    "https://www.example.com/product-2",
]

ingested_items: list[dict] = []
for url in product_urls:
    response = fashion_app.wardrobe_ingestion.ingest(user_id=user_id, urls=[url])
    ingested_items.extend(response.get("items", []))
    if response.get("failures"):
        print(f"Ingestion issues for {url}: {response['failures']}")

wardrobe_items_df = pd.DataFrame(ingested_items)
display(wardrobe_items_df)


In [ ]:
import pandas as pd
# Ingest wardrobe items directly from a pandas DataFrame. The schema aligns with models/wardrobe_item.py.
user_id = "notebook-demo-user"

manual_items_df = pd.DataFrame([
    {
        "item_id": "demo-top-001",
        "image_url": "https://example.com/images/placeholder-top.jpg",
        "source_url": "https://example.com/product/top-placeholder",
        "category": "top",
        "sub_category": "shirt",
        "colors": ["white"],
        "materials": ["cotton"],
        "brand": "Notebook Demo",
        "fit": "relaxed",
        "season_tags": ["all_year"],
        "style_tags": ["casual"],
        "user_notes": "Sample shirt entry from the notebook.",
    },
    {
        "item_id": "demo-sneaker-001",
        "image_url": "https://example.com/images/placeholder-sneaker.jpg",
        "source_url": "https://example.com/product/sneaker-placeholder",
        "category": "shoes",
        "sub_category": "sneakers",
        "colors": ["black", "white"],
        "materials": ["leather", "rubber"],
        "brand": "Notebook Demo",
        "fit": "relaxed",
        "season_tags": ["all_year"],
        "style_tags": ["casual", "street"],
        "user_notes": "Sample sneakers entry from the notebook.",
    },
])

persisted_items: list[dict] = []
for _, row in manual_items_df.iterrows():
    persisted = fashion_app.wardrobe_tools.add_wardrobe_item(
        user_id=user_id, item_data=row.to_dict()
    )
    persisted_items.append(persisted)

print(f"Persisted {len(persisted_items)} manual wardrobe items for {user_id}.")

stored_items = fashion_app.wardrobe_tools.list_wardrobe_items(user_id=user_id)
display(pd.DataFrame(stored_items))


## 3. End-to-end outfit suggestion demo

This section runs a **single full interaction**:

1. Create a fresh session.  
2. Ask the agent to plan outfits for a given day, location and mood.  
3. Inspect the structured result and any natural language rationale.

If your backend is wired as intended, this will exercise:

- the **calendar agent** (to infer formality and schedule),  
- the **weather agent** (to choose layers and fabrics), and  
- the **wardrobe / stylist agents** (to assemble and score outfit combinations).


### 3.0 Quick orchestrator smoke test

Before calling any richer outfit logic, we can run a very small test that sends
a message through the orchestrator path using `send_test_message`. This helps
confirm that sessions and the top level agent wiring are working.


In [17]:
# Quick orchestrator smoke test
smoke_session, smoke_session_id = create_demo_session(metadata={"demo": "orchestrator-smoke"})

smoke_query = (
    "This is a quick smoke test from the notebook. "
    "Summarise what you are and what capabilities you provide as a fashion concierge agent."
)

smoke_response = send_demo_message(session_id=smoke_session_id, user_query=smoke_query)
smoke_response


{"timestamp": "2025-11-27T16:30:12+0000", "level": "INFO", "logger": "agents.orchestrator", "message": "agent_call_started", "event": "agent_call_started", "correlation_id": "be1e38b5f4734a85828c503fc2b3c2af", "taskName": "Task-79", "agent": "orchestrator", "method": "handle_message", "session_id": null}
{"timestamp": "2025-11-27T16:30:12+0000", "level": "INFO", "logger": "agents.orchestrator", "message": "agent_call_completed", "event": "agent_call_completed", "correlation_id": "be1e38b5f4734a85828c503fc2b3c2af", "taskName": "Task-79", "agent": "orchestrator", "method": "handle_message", "session_id": null, "status": "unknown"}


Created session id: baf2556d-df34-4990-ab79-8787534591a1


'This is a scaffolded orchestrator. Expand sub-agent calls next.'

In [19]:
# 1. Create a new session for this demo
session, session_id = create_demo_session(metadata={"demo": "single-day-outfit"})

# 2. Define the high-level request for the orchestrator
user_query = (
    "I am in Rotterdam this Friday with a full workday and casual drinks after. "
    "Suggest one daytime outfit and one outfit that can transition into the evening. "
    "Keep the mood 'trendy' but practical for commuting by bike."
)

# Optional: you can specify a particular date if your backend expects it, e.g. "2025-11-28"
demo_date = None  # or "2025-11-28"

# 3. Call the orchestrator
outfit_response = orchestrate_outfit(
    session_id=session_id,
    user_query=user_query,
    location="Rotterdam, NL",
    date_iso=demo_date,
    mood="trendy",
)

outfit_response

{"timestamp": "2025-11-27T16:30:42+0000", "level": "INFO", "logger": "agents.orchestrator", "message": "agent_call_started", "event": "agent_call_started", "correlation_id": "be1e38b5f4734a85828c503fc2b3c2af", "taskName": "Task-85", "agent": "orchestrator", "method": "handle_message", "session_id": null}
{"timestamp": "2025-11-27T16:30:42+0000", "level": "INFO", "logger": "agents.orchestrator", "message": "agent_call_completed", "event": "agent_call_completed", "correlation_id": "be1e38b5f4734a85828c503fc2b3c2af", "taskName": "Task-85", "agent": "orchestrator", "method": "handle_message", "session_id": null, "status": "unknown"}


Created session id: 9809cb10-35ba-4cbb-920c-d2a95caa41fd


'This is a scaffolded orchestrator. Expand sub-agent calls next.'

### 3.1 Interpreting the result

The variable `outfit_response` may be:

- a plain string (just text),  
- a Pydantic model / dataclass, or  
- a nested dictionary with keys like `outfits`, `candidates`, `context`, etc.

For inspection and debugging it helps to **look at the raw object** and then
pull out key parts (for example, outfit items and their scores). The helper
below tries to do this in a best-effort way.


In [20]:
from pprint import pprint

def pretty_print_outfits(resp):
    """Best-effort pretty printer for common response shapes."""
    if resp is None:
        print("No response returned.")
        return

    obj = resp
    if hasattr(resp, "model_dump"):
        obj = resp.model_dump()
    elif hasattr(resp, "dict"):
        obj = resp.dict()

    if isinstance(obj, dict):
        print("Top-level keys:", list(obj.keys()))
        for key in ("outfits", "candidates", "suggestions"):
            if key in obj:
                print("\n===", key, "===")
                pprint(obj[key], depth=3)
                break
        else:
            pprint(obj, depth=3)
    else:
        print(obj)


pretty_print_outfits(outfit_response)

This is a scaffolded orchestrator. Expand sub-agent calls next.


## 3.1 Direct outfit stylist demo

In addition to the orchestrator smoke test, we can call the outfit stylist
agent directly. This focuses on the deterministic outfit building and scoring
logic without going through calendar or weather.

The exact method signature depends on `outfit_stylist_agent.py`. You can
adjust the helper below to match your implementation.


In [24]:
def demo_recommend_outfit_direct(
    user_id: str = "notebook-demo-user",
    mood: str = "trendy",
    constraints: Optional[list[str]] = None,
    schedule_profile: Optional[dict] = None,
    weather_profile: Optional[dict] = None,
    daily_context: Optional[dict] = None,
    top_n: int = 3,
):
    """Call the outfit stylist agent directly.

    This matches the actual signature of OutfitStylistAgent.recommend_outfit:

        (user_id: str,
         mood: Optional[str] = None,
         constraints: Optional[List[str]] = None,
         schedule_profile: Optional[Dict[str, object]] = None,
         weather_profile: Optional[Dict[str, object]] = None,
         daily_context: Optional[Dict[str, object]] = None,
         top_n: int = 3) -> Dict[str, object]

    For this demo we pass only user_id and mood and leave the other
    arguments as None so the stylist uses its internal defaults.
    """
    stylist = fashion_app.outfit_stylist
    print("Outfit stylist agent:", stylist)

    if not hasattr(stylist, "recommend_outfit"):
        raise AttributeError(
            "The outfit stylist agent does not expose 'recommend_outfit'. "
            "Open agents/outfit_stylist_agent.py and either expose one or "
            "update this helper to use the correct method."
        )

    outfit = stylist.recommend_outfit(
        user_id=user_id,
        mood=mood,
        constraints=constraints,
        schedule_profile=schedule_profile,
        weather_profile=weather_profile,
        daily_context=daily_context,
        top_n=top_n,
    )
    return outfit


# Example call for the direct stylist demo
direct_outfit = demo_recommend_outfit_direct(
    user_id="notebook-demo-user",
    mood="trendy",
)

pretty_print_outfits(direct_outfit)


{"timestamp": "2025-11-27T16:42:55+0000", "level": "INFO", "logger": "agents.outfit_stylist_agent", "message": "agent_call_started", "event": "agent_call_started", "correlation_id": "be1e38b5f4734a85828c503fc2b3c2af", "taskName": "Task-100", "agent": "stylist", "method": "recommend_outfit", "user_id": "[redacted]", "mood": "trendy"}
{"timestamp": "2025-11-27T16:42:55+0000", "level": "INFO", "logger": "tools.observability", "message": "tool_call_started", "event": "tool_call_started", "correlation_id": "be1e38b5f4734a85828c503fc2b3c2af", "taskName": "Task-100", "tool": "list_wardrobe_items", "kwargs": {}}
{"timestamp": "2025-11-27T16:42:55+0000", "level": "INFO", "logger": "tools.observability", "message": "tool_call_completed", "event": "tool_call_completed", "correlation_id": "be1e38b5f4734a85828c503fc2b3c2af", "taskName": "Task-100", "tool": "list_wardrobe_items", "duration_ms": 1.06}
{"timestamp": "2025-11-27T16:42:55+0000", "level": "INFO", "logger": "agents.outfit_stylist_agent", 

Outfit stylist agent: <agents.outfit_stylist_agent.OutfitStylistAgent object at 0x73f7977c8200>
Top-level keys: ['ranked_outfits', 'user_facing_rationale', 'debug_summary']
{'debug_summary': {'candidate_outfits': 0,
                   'daily_context': {'formality_requirement': 'informal',
                                     'movement_requirement': 'low',
                                     'special_constraints': [],
                                     'warmth_requirement': 'medium',
                                     'weather_risk_level': 'low'},
                   'filters': {'final_count': 0,
                               'items': [],
                               'reasons': {},
                               'steps': [...]},
                   'ranked_outfits': []},
 'ranked_outfits': [],
 'user_facing_rationale': 'Generated 0 trendy outfits using movement low and '
                          'formality informal.'}


## 4. Evaluation scenario demo

Here we run the outfit stylist on one of the predefined evaluation scenarios
from the repository. This demonstrates how the same logic can be used for
regression testing and quality checks.


In [ ]:
from evaluation.scenarios import SCENARIOS
from logic.context_synthesizer import synthesize_context

# Pick a scenario oriented toward daytime outfits; default to the first if none match.
scenario = next((s for s in SCENARIOS if s.mood in ("casual", "neutral", "happy")), SCENARIOS[0])

print(f"Using evaluation scenario: {scenario.name} – {scenario.description}")
print(f"Location: {scenario.location} | Mood: {scenario.mood} | Target date: {scenario.target_date}")

# Use a dedicated evaluation user id so items do not clash with other notebook demos.
eval_user_id = "notebook-eval-user"

# Seed the wardrobe for this scenario; re-running the cell simply refreshes the items.
for item in scenario.wardrobe_items:
    fashion_app.wardrobe_tools.add_wardrobe_item(user_id=eval_user_id, item_data=item)

def schedule_profile_from_events(events):
    """Lightweight mapping from scenario events to the stylist-friendly schedule profile."""
    if not events:
        return {"formality": "informal", "movement": "low", "day_parts": []}

    day_parts = []
    categories = []
    for event in events:
        title = (event.title or "").lower()
        hour = getattr(getattr(event, "start_time", None), "hour", None)
        if hour is not None:
            if hour < 12:
                day_parts.append("morning")
            elif hour < 17:
                day_parts.append("afternoon")
            else:
                day_parts.append("evening")

        if any(keyword in title for keyword in ("client", "meeting", "office")):
            categories.append("business")
        elif any(keyword in title for keyword in ("party", "social")):
            categories.append("social")
        elif any(keyword in title for keyword in ("gym", "workout", "travel", "commute", "airport", "flight")):
            categories.append("active")
        else:
            categories.append("casual")

    formality = "business" if "business" in categories else "informal"
    movement = "high" if any(cat == "active" for cat in categories) else ("medium" if "social" in categories else "low")

    return {"formality": formality, "movement": movement, "day_parts": sorted(set(day_parts))}

def weather_profile_from_scenario(profile):
    """Convert the scenario WeatherProfile into the labels the stylist expects."""
    if profile is None:
        # Fall back to mild/dry guidance if the scenario omits weather details.
        return {"layers_required": "one", "rain_sensitivity": "dry", "temperature_range": "mild"}

    avg_temp = (profile.temp_min + profile.temp_max) / 2.0
    if avg_temp < 5:
        temp_range = "cold"
    elif avg_temp < 12:
        temp_range = "cool"
    elif avg_temp < 18:
        temp_range = "mild"
    elif avg_temp < 24:
        temp_range = "warm"
    else:
        temp_range = "hot"

    rain = "heavy rain" if profile.precipitation_probability > 0.6 else (
        "light rain" if profile.precipitation_probability > 0.3 else "dry"
    )
    layers_required = {"cold": "two plus", "cool": "two", "mild": "one", "warm": "zero", "hot": "zero"}[temp_range]

    return {
        "layers_required": layers_required,
        "rain_sensitivity": rain,
        "temperature_range": temp_range,
        "raw_forecast": profile,
    }

schedule_profile = schedule_profile_from_events(scenario.calendar_events)
weather_profile = weather_profile_from_scenario(scenario.weather_profile)
daily_context = synthesize_context(schedule_profile, weather_profile)

eval_outfit = demo_recommend_outfit_direct(
    user_id=eval_user_id,
    mood=scenario.mood,
    schedule_profile=schedule_profile,
    weather_profile=weather_profile,
    daily_context=daily_context,
)

pretty_print_outfits(eval_outfit)


## 5. Session and memory demo

Now we demonstrate how sessions and memory work together.

1. Start a **new session**.  
2. First turn: the user describes a long-term style preference.  
3. Second turn: the user asks for an outfit; the agent should incorporate that preference.

This relies on your backend wiring a conversational entry point that:

- keeps **short-term state** in the ADK session, and  
- optionally writes **long-term preferences** into a memory store.


In [27]:
# Create a dedicated session for the memory demo and record the user id for memory lookups
memory_user_id = "notebook-demo-user"
memory_session, memory_session_id = create_demo_session(
    metadata={"demo": "memory"},
    user_id=memory_user_id,
)

# Turn 1 – the user states a preference
pref_query = "Remember that I prefer monochrome outfits with chunky sneakers and no logos."

# For now, we reuse `send_test_message` as a simple conversational stand in.
# The current implementation does not take a session id, so we just pass the text.
if hasattr(fashion_app, "send_test_message"):
    first_turn = fashion_app.send_test_message(pref_query)
else:
    raise AttributeError(
        "FashionConciergeApp does not expose a conversational entry point. "
        "Add one or update this cell once it exists."
    )

print("First turn response:
", first_turn, "
")

# Persist the stated preference into long-term memory for visibility in this demo.
if hasattr(fashion_app, "memory_service"):
    fashion_app.memory_service.update_user_preferences(
        user_id=memory_user_id,
        updates={"stated_preference": pref_query},
    )
else:
    raise AttributeError(
        "FashionConciergeApp does not expose a memory_service. "
        "Add one or update this cell once it exists."
    )



{"timestamp": "2025-11-27T16:46:16+0000", "level": "INFO", "logger": "agents.orchestrator", "message": "agent_call_started", "event": "agent_call_started", "correlation_id": "be1e38b5f4734a85828c503fc2b3c2af", "taskName": "Task-109", "agent": "orchestrator", "method": "handle_message", "session_id": null}
{"timestamp": "2025-11-27T16:46:16+0000", "level": "INFO", "logger": "agents.orchestrator", "message": "agent_call_completed", "event": "agent_call_completed", "correlation_id": "be1e38b5f4734a85828c503fc2b3c2af", "taskName": "Task-109", "agent": "orchestrator", "method": "handle_message", "session_id": null, "status": "unknown"}


Created session id: 058bb5fe-b3ef-4e94-ad59-bc23c4ea521b
First turn response:
 This is a scaffolded orchestrator. Expand sub-agent calls next. 



In [ ]:
from pprint import pprint

# Inspect the stored profile to confirm the preference was saved for this user.
memory_snapshot = fashion_app.memory_service.get_user_profile(user_id=memory_user_id)
print(f"Current stored profile for {memory_user_id}:")
pprint(memory_snapshot)

# If the orchestrator is not yet wiring conversational turns to memory,
# this demonstrates the available hooks; full integration remains a work in progress.


In [28]:
# Turn 2 – ask for a casual outfit; in a full implementation the agent
# would ideally remember the earlier preference.

followup_query = "Now suggest an outfit for a relaxed Sunday brunch in Amsterdam."

if hasattr(fashion_app, "send_test_message"):
    second_turn = fashion_app.send_test_message(followup_query)
else:
    raise AttributeError(
        "FashionConciergeApp does not expose a conversational entry point. "
        "Add one or update this cell once it exists."
    )

print("Second turn response:
", second_turn)

# Basic heuristic to see if the response acknowledges the stored preference.
if isinstance(second_turn, dict) and "outfits" in second_turn:
    combined_text = " ".join(str(outfit) for outfit in second_turn.get("outfits", []))
else:
    combined_text = str(second_turn)

preference_cues = ["monochrome", "chunky", "logo"]
matched_cues = [cue for cue in preference_cues if cue.lower() in combined_text.lower()]

if matched_cues:
    print(f"Preference cues present in response: {', '.join(matched_cues)}")
else:
    print(
        "Preference cues not detected in the response; the memory-aware stylist "
        "wiring may still be in progress."
    )



{"timestamp": "2025-11-27T16:46:39+0000", "level": "INFO", "logger": "agents.orchestrator", "message": "agent_call_started", "event": "agent_call_started", "correlation_id": "be1e38b5f4734a85828c503fc2b3c2af", "taskName": "Task-112", "agent": "orchestrator", "method": "handle_message", "session_id": null}
{"timestamp": "2025-11-27T16:46:39+0000", "level": "INFO", "logger": "agents.orchestrator", "message": "agent_call_completed", "event": "agent_call_completed", "correlation_id": "be1e38b5f4734a85828c503fc2b3c2af", "taskName": "Task-112", "agent": "orchestrator", "method": "handle_message", "session_id": null, "status": "unknown"}


Second turn response:
 This is a scaffolded orchestrator. Expand sub-agent calls next.


## 6. Where to go next

Once the basic flows above are working, you can extend this notebook to:

- Visualise **agent traces** (tool calls, reasoning steps) for a given session.  
- Call your **evaluation harness** to run predefined scenarios and compute scores.  
- Show how this notebook maps to the **HTTP API** in `server/api.py`, and how the same
  patterns apply when deploying to **Vertex AI Agent Engine** or **Cloud Run**.

Because this notebook is designed to run from inside the repository (including
GitHub Codespaces), you can commit it to version control and treat it as both:

- a **developer tool** for experimenting with the agent, and  
- a **readable artifact** for your capstone submission.
